In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input,models,layers,optimizers,metrics
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.applications import EfficientNetB7
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [2]:
cp = ModelCheckpoint(monitor = 'val_loss',filepath = "./models/Effi7.h5",save_best_only=True)
cp_ = ModelCheckpoint(monitor = 'val_accuracy',filepath = "./models/Effi7_acc.h5",save_best_only=True)
es = EarlyStopping(monitor = 'val_loss',patience=15)
lr = ReduceLROnPlateau(monitor = 'val_loss',patience=6)
optimizer = Adam(lr=0.02)


train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode='nearest')
# train_generator = train_datagen.flow_from_directory('C:/Users/Administrator/Downloads/mnist/dirty_mnist_2nd/',target_size=(150,150),batch_size=4,class_mode='binary')
test_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode='nearest')
# test_generator = test_datagen.flow_from_directory('C:/Users/Administrator/Downloads/mnist/test_dirty_mnist_2nd/',target_size=(150,150),batch_size=4,class_mode='binary')

transfer_model = EfficientNetB7(weights='imagenet',include_top=False,input_shape=(256,256,3))
transfer_model.trainable=False

model = models.Sequential()
model.add(transfer_model)
model.add(Flatten())
model.add(Dense(1000,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
def read_data(train=True):
    result = []
    if train:
        for i in range(50000):
            imgpath = "./dirty_mnist_2nd/{}.png".format(str((5-len(str(i)))*'0')+str(i))
            img = cv2.imread(imgpath,cv2.IMREAD_GRAYSCALE)
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            result.append(img)
        return np.array(result).reshape(-1,256,256,3)/255.0
    else:
        for i in range(5000):
            imgpath="./test_dirty_mnist_2nd/{}.png".format(str(50000+i))
            img = cv2.imread(imgpath,cv2.IMREAD_GRAYSCALE)
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            result.append(img)
        return np.array(result).reshape(-1,256,256,3)/255.0

In [ ]:
x_train = np.load('./npy/npy_x_train.npy')
y_train = np.load('./npy/npy_y_train.npy')
x_val = np.load('./npy/npy_x_val.npy')
y_val = np.load('./npy/npy_y_val.npy')

In [5]:
x_test = np.load("./npy/npy_x_test.npy")

In [ ]:
x_train.shape,x_val.shape,y_train.shape,y_val.shape

In [ ]:
train_generator = train_datagen.flow(x_train,y_train,batch_size=16)
val_generator = val_datagen.flow(x_val,y_val)

test_generator = test_datagen.flow(x_test)

In [3]:
x_train = np.load('./npy/npy_x_train.npy')
y_train = np.load('./npy/npy_y_train.npy')
x_val = np.load('./npy/npy_x_val.npy')
y_val = np.load('./npy/npy_y_val.npy')

In [ ]:

alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
for i in range(26):
    if i==0 or i==1:
        continue
    y_train_ = y_train[:,i]
    y_val_ = y_val[:,i]
    
    cp = ModelCheckpoint(monitor = 'val_loss',filepath = "./models/Effi7__{}.h5".format(alphabet[i]),save_best_only=True)
    cp_ = ModelCheckpoint(monitor = 'val_accuracy',filepath = "./models/Effi7_acc__{}.h5".format(alphabet[i]),save_best_only=True)
    es = EarlyStopping(monitor = 'val_loss',patience=15)
    lr = ReduceLROnPlateau(monitor = 'val_loss',patience=6)
    optimizer = Adam(lr=0.02)

    train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode='nearest')
    val_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode='nearest')
    # train_generator = train_datagen.flow_from_directory('C:/Users/Administrator/Downloads/mnist/dirty_mnist_2nd/',target_size=(150,150),batch_size=4,class_mode='binary')
    test_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode='nearest')
    # test_generator = test_datagen.flow_from_directory('C:/Users/Administrator/Downloads/mnist/test_dirty_mnist_2nd/',target_size=(150,150),batch_size=4,class_mode='binary')

    transfer_model = EfficientNetB7(weights='imagenet',include_top=False,input_shape=(256,256,3))
    transfer_model.trainable=False
    
    train_generator = train_datagen.flow(x_train,y_train_,batch_size=16)
    val_generator = val_datagen.flow(x_val,y_val_)
    
    model = models.Sequential()
    model.add(transfer_model)
    model.add(Flatten())
    model.add(Dense(1000,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit_generator(train_generator,epochs=8,verbose=1)
    del model
    
    

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/8
2657/2657 [==============================] - 400s 141ms/step - loss: 3.6229 - accuracy: 0.5230
Epoch 2/8
2657/2657 [==============================] - 319s 120ms/step - loss: 0.6957 - accuracy: 0.5390
Epoch 3/8
1555/2657 [================>.............] - ETA: 2:12 - loss: 0.7103 - accuracy: 0.5387

In [4]:
model = load_model("./models/Effi7.h5")
x_test = np.load("./npy/npy_x_test.npy")
y_pred = model.predict(x_test)
df = pd.read_csv("./sample_submission.csv",index_col=0)
df.values=y_pred
df.values

,a,b,c,d,e,f,g,h,i,j,...,q,r,s,t,u,v,w,x,y,z
index,,,,,,,,,,,,,,,,,,,,,
50000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
model = load_model("./models/Effi7.h5")
x_test = np.load("./npy/npy_x_test.npy")
y_pred = model.predict(x_test)
df = pd.read_csv("./sample_submission.csv",index_col=0)
df[:,:]=y_pred
df.values

<ipython-input-6-abe2ecf68396>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.values=y_pred


AttributeError: can't set attribute

In [8]:
df.iloc[:,:]=y_pred
df.values

array([[0.45798635, 0.46133569, 0.46174031, ..., 0.46098012, 0.46106768,
        0.45898083],
       [0.45798635, 0.46133569, 0.46174031, ..., 0.46098012, 0.46106768,
        0.45898083],
       [0.45798635, 0.46133569, 0.46174031, ..., 0.46098012, 0.46106768,
        0.45898083],
       ...,
       [0.45798635, 0.46133569, 0.46174031, ..., 0.46098012, 0.46106768,
        0.45898083],
       [0.45798635, 0.46133569, 0.46174031, ..., 0.46098012, 0.46106768,
        0.45898083],
       [0.45798635, 0.46133569, 0.46174031, ..., 0.46098012, 0.46106768,
        0.45898083]])